# KATL Final Approach Simulation
### ! conda activate tracon

In [1]:
import numpy as np
import pandas as pd
import math
import pickle

# save pickle full state
# st = np.random.get_state()
# with open("state1.pkl", "wb") as f:
#     pickle.dump(st, f)


# load pickle full state
with open("state1.pkl", "rb") as f:
    state = pickle.load(f)

# 2. Restore NumPy's RNG to that state
np.random.set_state(state)

# ==========================================
# 1. PARAMETERS & CONSTANTS
# ==========================================

# Simulation Constraints
R_TURN_NM = 6.0       # Radius of RF turn (nmi)
T_SEP_SEC = 66.0       # Minimum separation (seconds)
T_MAX_SEC = 3600.0     # Simulation duration (1 hour)

# Unit Conversions
KM_TO_NM = 0.539957    # 1 Kilometer = 0.539957 Nautical Miles
R_EARTH_KM = 6371.0    # Radius of Earth in KM

# Reference Point: KATL Runway 9L Threshold (Origin 0,0)
REF_POINT = {'lat': 33.634667, 'lon': -84.448000}

# Fix Definitions (Lat/Lon)
# Note: VINII is included here to calculate its relative X/Y position
raw_fixes = {
    'VINII': {'lat': 33.634650, 'lon': -84.549842, 'type': 'FAF', 'flow': 'FAF'},
    'DALAS': {'lat': 33.952250, 'lon': -84.848022, 'type': 'Corner', 'flow': 'NorthWest'},
    'LOGEN': {'lat': 33.988050, 'lon': -84.056786, 'type': 'Corner', 'flow': 'NorthEast'},
    'HUSKY': {'lat': 33.330458, 'lon': -83.980208, 'type': 'Corner', 'flow': 'SouthEast'},
    'TIROE': {'lat': 33.306453, 'lon': -84.866031, 'type': 'Corner', 'flow': 'SouthWest'}
}

# ==========================================
# 2. GEOMETRY FUNCTIONS (Great Circle)
# ==========================================

def get_distance_and_bearing(lat1, lon1, lat2, lon2):
    """
    Calculates Great Circle distance (km) and initial bearing (degrees)
    between two points.
    """
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    # --- Haversine Formula for Distance ---
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    dist_km = R_EARTH_KM * c
    
    # --- Bearing Formula ---
    y = math.sin(dlambda) * math.cos(phi2)
    x = math.cos(phi1) * math.sin(phi2) - math.sin(phi1) * math.cos(phi2) * math.cos(dlambda)
    bearing_rad = math.atan2(y, x)
    bearing_deg = (math.degrees(bearing_rad) + 360) % 360
    
    return dist_km, bearing_deg

def geo_to_cartesian(target_lat, target_lon, ref_lat, ref_lon):
    """
    Converts target Lat/Lon to X (East) / Y (North) in Nautical Miles
    relative to the reference point (0,0).
    """
    # 1. Get Distance (km) and Bearing (deg)
    dist_km, bearing_deg = get_distance_and_bearing(ref_lat, ref_lon, target_lat, target_lon)
    
    # 2. Convert to NM
    dist_nm = dist_km * KM_TO_NM
    
    # 3. Project to Cartesian (Standard Math: 0 deg = East, but Map: 0 deg = North)
    # Map Bearing: 0 is North (Y+), 90 is East (X+)
    bearing_rad = math.radians(bearing_deg)
    
    x_nm = dist_nm * math.sin(bearing_rad) # Sin gives East component for Map Bearing
    y_nm = dist_nm * math.cos(bearing_rad) # Cos gives North component for Map Bearing
    
    return x_nm, y_nm

# ==========================================
# 3. INITIALIZE GEOMETRY
# ==========================================

processed_fixes = {}

for name, data in raw_fixes.items():
    x, y = geo_to_cartesian(data['lat'], data['lon'], REF_POINT['lat'], REF_POINT['lon'])
    
    processed_fixes[name] = data.copy()
    processed_fixes[name]['x'] = x
    processed_fixes[name]['y'] = y
    processed_fixes[name]['dist (nmi)'] = np.sqrt(x**2 + y**2)
    processed_fixes[name]['flow'] = data['flow']

print("Geometry Setup Complete.")
print(f"Origin (Runway 9L Threshold): (0.0, 0.0) nm")
print(f"FAF (VINII) Calculated Location: ({processed_fixes['VINII']['x']:.4f}, {processed_fixes['VINII']['y']:.4f}) nm")
pd.DataFrame(processed_fixes).T[['type', 'x', 'y', 'dist (nmi)', 'flow']] # Display Table

Geometry Setup Complete.
Origin (Runway 9L Threshold): (0.0, 0.0) nm
FAF (VINII) Calculated Location: (-5.0910, 0.0015) nm


,type,x,y,dist (nmi),flow
VINII,FAF,-5.090967,0.001485,5.090968,FAF
DALAS,Corner,-19.922658,19.106464,27.603791,NorthWest
LOGEN,Corner,19.475811,21.254227,28.827927,NorthEast
HUSKY,Corner,23.466697,-18.211933,29.704551,SouthEast
TIROE,Corner,-20.976252,-19.663862,28.751881,SouthWest


In [2]:
# ==========================================
# 4. ARRIVAL GENERATION FUNCTIONS
# ==========================================

def generate_shifted_poisson_times(lambda_ph, t_sep, t_max):
    """
    Generates a list of arrival times (seconds) based on Shifted Poisson Process.
    Delta_t = T_sep + Exp(1/Lambda)
    """
    lambda_per_sec = lambda_ph / 3600.0
    arrivals = []
    t = 0.0
    
    while t < t_max:
        # Stochastic component: Exponential distribution
        stochastic_wait = np.random.exponential(1.0 / lambda_per_sec)
        
        # Total interval: Hard separation + Stochastic
        inter_arrival = t_sep + stochastic_wait
        
        t += inter_arrival
        if t > t_max:
            break
        arrivals.append(t)
        
    return arrivals

def generate_scenario(seed=None):
    np.random.seed(seed)
    all_flights = []
    
    # Retrieve FAF location for Logic checks
    faf_x = processed_fixes['VINII']['x']
    faf_y = processed_fixes['VINII']['y']

    # Iterate only through the 4 corners (exclude VINII from being a start point)
    corners = [k for k, v in processed_fixes.items() if v['type'] == 'Corner']
    
    print(f"{'Corner':<10} | {'Lambda (arr/hr)':<15} | {'Count'}")
    print("-" * 40)

    for name in corners:
        fix_data = processed_fixes[name]
        
        # 1. Sample Demand (Log-Uniform between 5 and 45 aircraft/hr)
        # This ensures we test both low and high traffic scenarios
        log_lambda = np.random.uniform(np.log(5), np.log(15))
        lambda_s = np.exp(log_lambda)
        
        # 2. Generate Times
        times = generate_shifted_poisson_times(lambda_s, T_SEP_SEC, T_MAX_SEC)
        
        # 3. Determine Geometry Logic
        # North/South Logic (for turn direction center calculation later)
        is_north = fix_data['y'] > faf_y
        
        # LongArc Logic: 
        # If Fix X > FAF X, plane is "downwind" relative to FAF -> LongArc
        is_long_arc = 1 if fix_data['x'] > faf_x else 0
        
        print(f"{name:<10} | {lambda_s:>15.2f} | {len(times)}")
        
        # 4. Store Data
        for t in times:
            all_flights.append({
                'entry_time': t,
                'corner': fix_data['flow'],
                'enter_fix_name': name,
                # Entry State
                'x_entry': fix_data['x'],
                'y_entry': fix_data['y'],
                # Target State (FAF)
                'x_faf': faf_x,
                'y_faf': faf_y,
                # Optimization Flags
                'r_turn': R_TURN_NM,
                'is_north': is_north,
                'long_arc': is_long_arc
            })
            
    # Create DataFrame and sort by time
    df = pd.DataFrame(all_flights).sort_values('entry_time').reset_index(drop=True)
    df['aircraft_id'] = df.index + 1
    
    return df

# ==========================================
# 5. EXECUTE GENERATION
# ==========================================

df_arrivals = generate_scenario(seed=None)

print("\nSimulation Generated.")
print(f"Total Aircraft: {len(df_arrivals)}")
print("\nFirst 10 Arrivals:")
display(df_arrivals)

Corner     | Lambda (arr/hr) | Count
----------------------------------------
DALAS      |            5.55 | 8
LOGEN      |            5.21 | 4
HUSKY      |            8.04 | 3
TIROE      |            5.04 | 3

Simulation Generated.
Total Aircraft: 18

First 10 Arrivals:


,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id
0,78.282298,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1
1,104.472722,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,2
2,512.153815,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,3
3,670.531560,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4
4,722.692329,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,5
5,729.075953,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,6
6,1296.869124,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,7
7,1625.696037,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,8
8,1688.719910,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,9
9,2353.999406,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,10


# Pyomo Optimization

# Setup I: Optimize $d_i$ Only with FCFS (Fixed Sequence)

In [3]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# ==========================================
# 6. OPTIMIZATION MODEL (Fixed Sequence)
# ==========================================

def solve_schedule_optimization(df_arrivals, debug=False):
    """
    Builds and solves the NLP model to find optimal d_i (extension)
    to minimize makespan while satisfying T_sep.
    """
    
    # --- A. Setup Model ---
    m = pyo.ConcreteModel()
    
    # Indices (Aircraft IDs)
    # We assume the dataframe is already sorted by entry time (FCFS)
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Parameters (Fixed Data from DF)
    # We use dictionaries to map ID -> Value for Pyomo
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Global Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # Speeds (Knots) - Fixed for this formulation
    # v_L: Tangent leg, v_theta: Turn, v_f: Final straight
    v_L = 210.0
    v_theta = 180.0
    v_f = 150.0
    
    # Bounds for d_i (The extension leg)
    # Min 0.0, Max 20.0 nm (approx 37km extension limit)
    d_min, d_max = 0.0, 20.0 
    
    # --- B. Decision Variables ---
    # d[i]: The extension distance from FAF (nautical miles)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(d_min, d_max), initialize=0.0)
    
    # t[i]: The arrival time at FAF (seconds)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 500)

    # --- C. Geometry Expressions (The Math) ---
    # We define these as expressions so Pyomo can calculate derivatives
    
    def calc_travel_time_expr(m, i):
        # 1. Determine Center of Turn C0 based on d[i]
        # If North Arrival: Center is (X_faf - d, Y_faf + r)
        # If South Arrival: Center is (X_faf - d, Y_faf - r)
        # We use standard python if because 'p_is_north' is a parameter (constant)
        
        y_center_offset = r if p_is_north[i] else -r
        
        # Coordinates of Center C0
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        
        # Coordinates of Projected Center C0_prime
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        # 2. Calculate Distances d0 and d0_prime
        # Distance from Entry (Px, Py) to Center C0
        # pyo.sqrt is required for variables
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq)
        
        # Distance from Entry to Projected Center C0_prime
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # 3. Tangent Distance d_L (Pythagoras)
        # d_L = sqrt(d0^2 - r^2)
        # Small epsilon 1e-6 added to prevent sqrt(0) error if d0=r
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # 4. Angle Calculation (Thetas)
        # theta1 = acos((r^2 + d0^2 - d_L^2) / (2*r*d0)) -> Simplifies to acos(r/d0)
        # theta2 = acos((r^2 + d0^2 - d0_prime^2) / (2*r*d0))
        
        # We clamp inputs to acos to [-1, 1] to prevent numerical errors
        term1 = r / d0
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * d0)
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        # 5. Total Turn Angle theta
        if p_long_arc[i]:
            # Downwind: 2*pi - (t1 + t2)
            theta_radians = 2*3.14159 - (theta1 + theta2)
        else:
            # Base/Straight: t2 - t1
            theta_radians = theta2 - theta1
            
        # 6. Distances
        dist_turn = r * theta_radians
        dist_final = m.d[i] # The straight-in extension
        
        # 7. Total Time (in Seconds)
        # Distance (nm) / Speed (knots) * 3600 (sec/hr)
        time_L = (d_L / v_L) * 3600
        time_turn = (dist_turn / v_theta) * 3600
        time_final = (dist_final / v_f) * 3600
        
        return p_tau[i] + time_L + time_turn + time_final

    # --- D. Constraints ---
    
    # 1. Link Physics to Time Variable
    def physics_rule(m, i):
        return m.t[i] == calc_travel_time_expr(m, i)
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. Separation Constraint (Sequence Preserved)
    def separation_rule(m, i):
        # Skip the first aircraft
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # --- E. Objective ---
    # Minimize the arrival time of the LAST aircraft (Makespan)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id], sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    solver.options['max_iter'] = 10000
    if not debug:
        solver.options['print_level'] = 0 # Suppress solver text
        
    result = solver.solve(m, tee=debug)
    
    # --- G. Extract Results ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time_faf': pyo.value(m.t[i]),
            'scheduled_entry': p_tau[i]
        })
        
    return pd.DataFrame(results)

# ==========================================
# 7. RUN OPTIMIZATION
# ==========================================

print("Optimizing Schedule...")
df_opt_results = solve_schedule_optimization(df_arrivals, debug=True)

# Merge results back with original info for viewing
df_final = pd.merge(df_arrivals, df_opt_results, on='aircraft_id')

# Calculate Delay (Arrival Time - Minimum Possible Time)
# Note: This is simplified; true delay is typically (Actual - Scheduled)
# Here we just look at the timeline.
df_final['transit_time'] = df_final['arrival_time_faf'] - df_final['entry_time']

print("\nOptimization Complete.")
display(df_final[['aircraft_id', 'corner', 'entry_time', 'optimized_d_i', 'arrival_time_faf', 'transit_time']])

Optimizing Schedule...
Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       36
Number of nonzeros in inequality constraint Jacobian.:       34
Number of nonzeros in Lagrangian Hessian.............:       18

Total number of variables............................:       36
                     variables with only lower bounds:       18
                variables with lower and upper bounds:       18
                     variables with only upper bounds:        0
Total number of equality constraints..

,aircraft_id,corner,entry_time,optimized_d_i,arrival_time_faf,transit_time
0,1,NorthWest,78.282298,1.151371e+00,544.300785,466.018487
1,2,SouthWest,104.472722,4.838951e+00,652.465830,547.993108
2,3,NorthWest,512.153815,8.289447e+00,1123.741585,611.587770
3,4,NorthWest,670.531560,1.132035e+01,1362.873428,692.341867
4,5,SouthEast,722.692329,-9.999978e-09,1588.068901,865.376572
5,6,SouthWest,729.075953,2.000000e+01,1644.849227,915.773274
6,7,NorthWest,1296.869124,8.729920e+00,1919.413410,622.544286
7,8,SouthWest,1625.696037,2.992451e+00,2139.462953,513.766915
8,9,NorthWest,1688.719910,7.326423e+00,2277.246833,588.526923
9,10,NorthEast,2353.999406,9.244960e-01,3211.629814,857.630408


## Introducing Slack Variables for High Density Scenarios

In [4]:
# ==========================================
# 6. OPTIMIZATION MODEL (Fixed Speed + Slack)
# ==========================================

def solve_schedule_optimization_fixed_speed_robust(df_arrivals, debug=False):
    """
    Implements Section 2.2: Optimize d_i only.
    Includes Slack Variables to handle infeasibility during high traffic.
    """
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Dictionaries for fast lookup
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # FIXED SPEEDS (Knots) - Constants, not Variables
    v_L_const = 210.0
    v_theta_const = 180.0
    v_f_const = 150.0
    
    # --- B. Decision Variables ---
    
    # 1. Path Extension (The only control variable)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    
    # 2. Arrival Time (Dependent variable)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 500)

    # 3. Slack Variable (The "Safety Valve")
    # Represents seconds of violation allowed to prevent solver crash
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # --- C. Geometry Expressions ---
    def calc_travel_time_expr(m, i):
        # Center Calculation
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        # Distances (d0, d0_prime)
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6)
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # Tangent Leg (d_L)
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # Angles
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        # Distances
        dist_turn = r * theta_rad
        dist_final = m.d[i]
        
        # Time Calculation using FIXED SPEEDS
        # (nm / knots) * 3600 = seconds
        t_L = (d_L / v_L_const) * 3600
        t_turn = (dist_turn / v_theta_const) * 3600
        t_final = (dist_final / v_f_const) * 3600
        
        return p_tau[i] + t_L + t_turn + t_final

    # --- D. Constraints ---
    
    # 1. Physics Constraint
    def physics_rule(m, i):
        return m.t[i] == calc_travel_time_expr(m, i)
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. Robust Separation Constraint
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        
        # Constraint: Time[i] >= Time[i-1] + 64 - Slack[i]
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # --- E. Objective ---
    # Minimize Last Arrival Time + Penalty for Violations
    # Weight = 1000. This means the solver will do EVERYTHING possible to extend d_i
    # before it dares to use 1 second of slack.
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 1000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    if not debug: solver.options['print_level'] = 0
    
    # Ipopt options for robustness
    solver.options['max_iter'] = 10000
    solver.options['tol'] = 1e-6
    
    solver.solve(m, tee=debug)
    
    # --- G. Extract Results ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'entry_time': p_tau[i],
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time_faf': pyo.value(m.t[i]),
            'scheduled_delay': pyo.value(m.t[i]) - p_tau[i]
        })
        
    return pd.DataFrame(results)

# --- Run Optimization ---
print("Running Robust Optimization (Fixed Speed, d_i only)...")
df_results = solve_schedule_optimization_fixed_speed_robust(df_arrivals, debug=True)

# Merge and Display
df_final = pd.merge(df_arrivals, df_results[['aircraft_id', 'optimized_d_i', 'arrival_time_faf']], on='aircraft_id')

# Calculate achieved separation
df_final['prev_arrival'] = df_final['arrival_time_faf'].shift(1)
df_final['actual_separation'] = df_final['arrival_time_faf'] - df_final['prev_arrival']

# Check total violation
if df_final['actual_separation'].any() < T_SEP_SEC:
    print("WARNING: Traffic demand exceeded capacity. Some aircraft were not separated by 64s.")
else:
    print("SUCCESS: All aircraft separated by 64s using only path extension.")

df_final

Running Robust Optimization (Fixed Speed, d_i only)...
Ipopt 3.14.19: max_iter=10000
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       36
Number of nonzeros in inequality constraint Jacobian.:       51
Number of nonzeros in Lagrangian Hessian.............:       18

Total number of variables............................:       54
                     variables with only lower bounds:       36
                variables with lower and upper bounds:       18
                     variables with only upper bounds:      

,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,optimized_d_i,arrival_time_faf,prev_arrival,actual_separation
0,78.282298,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1,5.890595e+00,634.610427,NaN,NaN
1,104.472722,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,2,1.345092e+01,862.847156,634.610427,228.236729
2,512.153815,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,3,9.015480e+00,1141.940973,862.847156,279.093817
3,670.531560,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.142054e+01,1365.758130,1141.940973,223.817157
4,722.692329,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,5,-9.977764e-09,1596.937973,1365.758130,231.179843
5,729.075953,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,6,2.000000e+01,1636.524365,1596.937973,39.586392
6,1296.869124,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,7,1.016582e+01,1956.962138,1636.524365,320.437773
7,1625.696037,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,8,8.631151e+00,2255.035315,1956.962138,298.073177
8,1688.719910,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,9,1.483108e+01,2489.908541,2255.035315,234.873226
9,2353.999406,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,10,3.931725e+00,3333.273464,2489.908541,843.364924


# Setup II: Co-optimize $d_i$ and segments speed 

In [5]:
# ==========================================
# 7. CO-OPTIMIZATION MODEL (Hard Constraints)
#    Variables: d_i, v_L, v_theta, v_f
#    Constraint: STRICT 64s separation
# ==========================================

def solve_co_optimization_no_slack(df_arrivals, debug=False):
    """
    Implements Section 2.3: Co-optimize d_i and Speeds.
    NO Slack variables. Enforces strict T_sep.
    May result in 'Infeasible' if traffic demand is too high.
    """
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Dictionaries
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # --- B. Decision Variables ---
    
    # 1. Path Extension (0 to 20 nm)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    
    # 2. Arrival Time (seconds)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    
    # 3. Variable Speeds (Knots) - With Bounds from your sets
    # We initialize to higher speeds to encourage "fastest possible" flow first
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)

    # --- C. Geometry Expressions ---
    def calc_distances_expr(m, i):
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6)
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        return d_L, d_theta, d_final

    # --- D. Constraints ---
    
    # 1. Physics: Time depends on Distance and Variable Speeds
    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. HARD Separation Constraint (Strict Inequality)
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        # STRICT: Current time must be >= Previous time + 64s
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # 3. Speed Monotonicity Constraints (v_L >= v_theta >= v_f)
    def speed_mono_1(m, i):
        return m.v_L[i] >= m.v_theta[i]
    m.c_speed_1 = pyo.Constraint(m.I, rule=speed_mono_1)

    def speed_mono_2(m, i):
        return m.v_theta[i] >= m.v_f[i]
    m.c_speed_2 = pyo.Constraint(m.I, rule=speed_mono_2)
    
    # --- E. Objective ---
    # Minimize Makespan (Arrival time of the last aircraft)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id], sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    # Increase iter limit because finding a feasible point with hard constraints is harder
    solver.options['max_iter'] = 10000
    
    if not debug:
        solver.options['print_level'] = 0
        
    results_obj = solver.solve(m, tee=debug)
    
    # Check if optimal
    status = results_obj.solver.termination_condition
    print(f"Solver Status: {status}")

    # --- G. Extract ---
    results = []
    # Even if infeasible, we try to extract values to see where it failed
    try:
        for i in m.I:
            results.append({
                'aircraft_id': i,
                'entry_time': p_tau[i],
                'optimized_d_i': pyo.value(m.d[i]),
                'arrival_time_faf': pyo.value(m.t[i]),
                'v_L': pyo.value(m.v_L[i]),
                'v_theta': pyo.value(m.v_theta[i]),
                'v_f': pyo.value(m.v_f[i])
            })
    except ValueError:
        print("Could not extract values (Solver likely failed completely).")
        return pd.DataFrame()
        
    return pd.DataFrame(results)

# --- Run ---
print("Running Co-Optimization (Hard Constraints)...")
df_hard_results = solve_co_optimization_no_slack(df_arrivals, debug=True)

if not df_hard_results.empty:
    df_final_hard = pd.merge(df_arrivals, df_hard_results, on='aircraft_id')
    # Calculate achieved separation
    df_final_hard['prev_arrival'] = df_final_hard['arrival_time_faf'].shift(1)
    df_final_hard['separation'] = df_final_hard['arrival_time_faf'] - df_final_hard['prev_arrival']
    
    display(df_final_hard[['aircraft_id', 'optimized_d_i', 'v_L', 'v_f', 'arrival_time_faf', 'separation']])
else:
    print("Optimization failed to find a feasible solution.")

Running Co-Optimization (Hard Constraints)...


Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       90
Number of nonzeros in inequality constraint Jacobian.:      106
Number of nonzeros in Lagrangian Hessian.............:      126

Total number of variables............................:       90
                     variables with only lower bounds:       18
                variables with lower and upper bounds:       72
                     variables with only upper bounds:        0
Total number of equality constraints.................:       

,aircraft_id,optimized_d_i,v_L,v_f,arrival_time_faf,separation
0,1,5.937218e+00,227.704225,149.182162,620.085659,NaN
1,2,1.315128e+01,224.823059,146.457707,855.616514,235.530855
2,3,8.906918e+00,226.202117,148.173672,1130.908386,275.291872
3,4,1.101948e+01,225.550109,147.440395,1351.067181,220.158795
4,5,1.135222e+00,237.434924,151.532790,1570.528720,219.461539
5,6,1.807797e+01,211.333699,136.731507,1671.622904,101.094184
6,7,1.063712e+01,225.658948,147.581210,1965.732534,294.109629
7,8,9.104787e+00,226.167460,148.122256,2256.979560,291.247026
8,9,1.483025e+01,223.791790,144.885323,2502.075873,245.096313
9,10,4.047411e+00,229.821202,149.381494,3295.029560,792.953687


### With Slack Variables

In [6]:
def solve_schedule_optimization_robust(df_arrivals, debug=False):
    m = pyo.ConcreteModel()
    
    # Sets & Parameters (Same as before)
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # --- Decision Variables ---
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
    
    # NEW: Slack Variable (Amount of separation violation allowed)
    # We initialize it to 0.0. If the solver uses this, it admits defeat on separation.
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # --- Expressions ---
    def calc_distances_expr(m, i):
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6) # Safety epsilon
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # Tangent d_L
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # Angles
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        return d_L, d_theta, d_final

    # --- Constraints ---
    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # NEW: Robust Separation Rule
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        
        # Constraint: t[i] must be at least t[i-1] + 64, 
        # BUT we subtract slack[i]. If slack[i] is positive, we allow a violation.
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
        
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # Speed constraints (same as before)
    m.c_speed_1 = pyo.Constraint(m.I, rule=lambda m, i: m.v_L[i] >= m.v_theta[i])
    m.c_speed_2 = pyo.Constraint(m.I, rule=lambda m, i: m.v_theta[i] >= m.v_f[i])
    
    # --- NEW Objective ---
    # Minimize Makespan + Huge Penalty for violating separation
    # Penalty weight = 1000 (1 second of violation costs as much as 1000 seconds of flight time)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 1000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # --- Solve ---
    solver = SolverFactory('ipopt')
    if not debug: solver.options['print_level'] = 0
    
    # Increase max iterations to handle the larger search space
    solver.options['max_iter'] = 10000 
    solver.solve(m, tee=debug)
    
    # --- Extract ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'd_i': pyo.value(m.d[i]),
            'arrival_time': pyo.value(m.t[i]),
            'v_L': pyo.value(m.v_L[i]),
            'v_theta': pyo.value(m.v_theta[i]),
            'v_f': pyo.value(m.v_f[i])
        })
    return pd.DataFrame(results)

print("Optimizing Schedule...")
df_opt_results = solve_schedule_optimization_robust(df_arrivals, debug=True)
df_final = pd.merge(df_arrivals, df_opt_results, on='aircraft_id')

# Calculate achieved separation
df_final['prev_arrival'] = df_final['arrival_time'].shift(1)
df_final['actual_separation'] = df_final['arrival_time'] - df_final['prev_arrival']

df_final

Optimizing Schedule...
Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       90
Number of nonzeros in inequality constraint Jacobian.:      123
Number of nonzeros in Lagrangian Hessian.............:      126

Total number of variables............................:      108
                     variables with only lower bounds:       36
                variables with lower and upper bounds:       72
                     variables with only upper bounds:        0
Total number of equality constraints..

,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,d_i,arrival_time,v_L,v_theta,v_f,prev_arrival,actual_separation
0,78.282298,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1,5.937218e+00,620.085659,227.704225,176.519612,149.182162,NaN,NaN
1,104.472722,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,2,1.315128e+01,855.616514,224.823059,173.438125,146.457707,620.085659,235.530855
2,512.153815,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,3,8.906918e+00,1130.908386,226.202117,175.231215,148.173672,855.616514,275.291872
3,670.531560,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.101948e+01,1351.067181,225.550109,174.435699,147.440395,1130.908386,220.158795
4,722.692329,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,5,1.135222e+00,1570.528720,237.434924,185.295514,151.532790,1351.067181,219.461539
5,729.075953,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,6,1.807797e+01,1671.622904,211.333699,158.108307,136.731507,1570.528720,101.094184
6,1296.869124,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,7,1.063712e+01,1965.732534,225.658948,174.584230,147.581210,1671.622904,294.109629
7,1625.696037,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,8,9.104787e+00,2256.979560,226.167460,175.166253,148.122256,1965.732534,291.247026
8,1688.719910,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,9,1.483025e+01,2502.075873,223.791790,171.656976,144.885323,2256.979560,245.096313
9,2353.999406,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,10,4.047411e+00,3295.029560,229.821202,178.023946,149.381494,2502.075873,792.953687


In [7]:
# Calculate Delay (Arrival Time - Minimum Possible Time)
# Note: This is simplified; true delay is typically (Actual - Scheduled)
# Here we just look at the timeline.
df_final['transit_time'] = df_final['arrival_time'] - df_final['entry_time']

print("\nOptimization Complete.")
display(df_final[['aircraft_id', 'corner', 'entry_time', 'd_i', 'arrival_time', 'transit_time']])


Optimization Complete.


,aircraft_id,corner,entry_time,d_i,arrival_time,transit_time
0,1,NorthWest,78.282298,5.937218e+00,620.085659,541.803361
1,2,SouthWest,104.472722,1.315128e+01,855.616514,751.143792
2,3,NorthWest,512.153815,8.906918e+00,1130.908386,618.754571
3,4,NorthWest,670.531560,1.101948e+01,1351.067181,680.535621
4,5,SouthEast,722.692329,1.135222e+00,1570.528720,847.836391
5,6,SouthWest,729.075953,1.807797e+01,1671.622904,942.546952
6,7,NorthWest,1296.869124,1.063712e+01,1965.732534,668.863409
7,8,SouthWest,1625.696037,9.104787e+00,2256.979560,631.283523
8,9,NorthWest,1688.719910,1.483025e+01,2502.075873,813.355963
9,10,NorthEast,2353.999406,4.047411e+00,3295.029560,941.030154


# Two Stage Optimization
 - To enforce the segment speeds only from speed option sets: 

$\mathcal V_L=\{180,190,200,210,220,230,240,250\} \\
\mathcal V_{\theta}=\{130,140,150,160,170,180,190,200\} \\
\mathcal V_F=\{130,140,150,160,170,180,190,200\} \\$

In [8]:
def solve_two_stage_discrete(df_arrivals, debug=False):
    print("--- STAGE 1: Solving Continuous Relaxation ---")
    # 1. Solve with continuous speeds (using the function we wrote previously)
    # Note: Using the robust version (with slack) ensures we get a solution to round
    df_stage1 = solve_schedule_optimization_robust(df_arrivals, debug=False)
    
    if df_stage1.empty:
        print("Stage 1 failed. Cannot proceed.")
        return pd.DataFrame()
        
    print("--- Rounding Speeds ---")
    # 2. Round speeds to nearest 10
    def round_to_10(x): return 10 * round(x / 10)
    
    # Create a dictionary of FIXED speeds for Stage 2
    fixed_v_L = {row['aircraft_id']: round_to_10(row['v_L']) for _, row in df_stage1.iterrows()}
    fixed_v_theta = {row['aircraft_id']: round_to_10(row['v_theta']) for _, row in df_stage1.iterrows()}
    fixed_v_f = {row['aircraft_id']: round_to_10(row['v_f']) for _, row in df_stage1.iterrows()}
    
    # Note: You can do logic here. E.g., "If rounding up violates speed limit, round down"
    # For now, we assume simple rounding keeps us within [180, 250] approx.
    
    print("--- STAGE 2: Re-optimizing d_i with Fixed Discrete Speeds ---")
    
    # 3. Build Stage 2 Model (Similar to "Fixed Speed" model, but speeds vary per aircraft)
    m = pyo.ConcreteModel()
    m.I = pyo.Set(initialize=df_arrivals['aircraft_id'].tolist())
    
    # Parameters
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # GEOMETRY CONSTANTS
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # VARIABLES
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals)
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0) # Keep slack for safety

    # EXPRESSIONS & CONSTRAINTS
    def physics_rule(m, i):
        # --- Geometry Calc (Same as before) ---
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        d0 = pyo.sqrt((p_x[i]-Cx)**2 + (p_y[i]-Cy)**2 + 1e-6)
        d0_p = pyo.sqrt((p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2)
        d_L = pyo.sqrt(d0**2 - r**2 + 1e-6)
        
        theta1 = pyo.acos(r/(d0+1e-6))
        theta2 = pyo.acos((r**2 + d0**2 - d0_p**2)/(2*r*(d0+1e-6)))
        theta = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        
        # --- TIME CALC USING FIXED ROUNDED SPEEDS ---
        # We use the fixed dictionary here!
        v_L_val = fixed_v_L[i]  
        v_theta_val = fixed_v_theta[i]
        v_f_val = fixed_v_f[i]
        
        # If you optimized all 3, round all 3 and put them here
        
        return m.t[i] == p_tau[i] + (d_L/v_L_val)*3600 + ((r*theta)/v_theta_val)*3600 + (m.d[i]/v_f_val)*3600

    m.phys = pyo.Constraint(m.I, rule=physics_rule)
    
    def sep_rule(m, i):
        prev = i-1
        if prev not in m.I: return pyo.Constraint.Skip
        return m.t[i] >= m.t[prev] + 64.0 - m.slack[i]
    m.sep = pyo.Constraint(m.I, rule=sep_rule)
    
    # Objective: Minimize Makespan + Penalty
    last = m.I.last()
    m.obj = pyo.Objective(expr=m.t[last] + 1000*sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # SOLVE
    solver = SolverFactory('ipopt')
    solver.options['print_level'] = 0
    solver.solve(m)
    
    # Extract
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'v_L_discrete': fixed_v_L[i], # This is now an integer multiple of 10
            'v_theta_discrete': fixed_v_theta[i],
            'v_f_discrete': fixed_v_f[i],
            'arrival_time': pyo.value(m.t[i])
        })
    return pd.DataFrame(res)

# Run it
df_discrete = solve_two_stage_discrete(df_arrivals)

# Calculate achieved separation
df_discrete = pd.merge(df_arrivals, df_discrete, on='aircraft_id')
df_discrete['prev_arrival'] = df_discrete['arrival_time'].shift(1)
df_discrete['actual_separation'] = df_discrete['arrival_time'] - df_discrete['prev_arrival']

display(df_discrete)

--- STAGE 1: Solving Continuous Relaxation ---
--- Rounding Speeds ---
--- STAGE 2: Re-optimizing d_i with Fixed Discrete Speeds ---


,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,optimized_d_i,v_L_discrete,v_theta_discrete,v_f_discrete,arrival_time,prev_arrival,actual_separation
0,78.282298,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,1,6.014726e+00,230,180,150,615.514694,NaN,NaN
1,104.472722,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,2,1.296653e+01,220,170,150,851.413103,615.514694,235.898409
2,512.153815,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,3,9.038531e+00,230,180,150,1123.134328,851.413103,271.721225
3,670.531560,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.082239e+01,230,170,150,1342.562973,1123.134328,219.428645
4,722.692329,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,5,1.157905e+00,240,190,150,1558.090319,1342.562973,215.527346
5,729.075953,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,6,1.801382e+01,210,160,140,1657.680637,1558.090319,99.590318
6,1296.869124,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,7,1.055166e+01,230,170,150,1961.070911,1657.680637,303.390274
7,1625.696037,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,8,9.602414e+00,230,180,150,2258.050103,1961.070911,296.979193
8,1688.719910,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,9,1.482776e+01,220,170,140,2520.597657,2258.050103,262.547554
9,2353.999406,NorthEast,LOGEN,19.475811,21.254227,-5.090967,0.001485,6.0,True,1,10,4.087770e+00,230,180,150,3292.040512,2520.597657,771.442855


## MINLP formulation for intervaled segment speed
 - Introduce the binary variables to select exactly one speed for each segment

In [9]:
# ==========================================
# 8. ROBUST MINLP EXACT FORMULATION
#    Discrete Speeds + Slack Variables
# ==========================================

def solve_exact_discrete_minlp_robust(df_arrivals, debug=False):
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Parameters
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # Discrete Speed Sets
    Set_VL = [180.0, 190.0, 200.0, 210.0, 220.0, 230.0, 240.0, 250.0]
    Set_Vtheta = [130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0]
    Set_VF = [130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0]
    
    m.Set_VL = pyo.Set(initialize=Set_VL)
    m.Set_Vtheta = pyo.Set(initialize=Set_Vtheta)
    m.Set_VF = pyo.Set(initialize=Set_VF)

    # --- B. Decision Variables ---
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    
    # CRITICAL: Slack Variable to prevent "Cycling/Infeasible" errors
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # Binary Speed Selection
    m.z_L = pyo.Var(m.I, m.Set_VL, domain=pyo.Binary)
    m.z_theta = pyo.Var(m.I, m.Set_Vtheta, domain=pyo.Binary)
    m.z_F = pyo.Var(m.I, m.Set_VF, domain=pyo.Binary)

    # --- C. Constraints ---

    # 1. Select Exactly One
    m.c_sel_L = pyo.Constraint(m.I, rule=lambda m, i: sum(m.z_L[i, v] for v in m.Set_VL) == 1)
    m.c_sel_theta = pyo.Constraint(m.I, rule=lambda m, i: sum(m.z_theta[i, v] for v in m.Set_Vtheta) == 1)
    m.c_sel_F = pyo.Constraint(m.I, rule=lambda m, i: sum(m.z_F[i, v] for v in m.Set_VF) == 1)

    # 2. Effective Speed Expressions
    def get_vL(m, i): return sum(v * m.z_L[i, v] for v in m.Set_VL)
    def get_vTheta(m, i): return sum(v * m.z_theta[i, v] for v in m.Set_Vtheta)
    def get_vF(m, i): return sum(v * m.z_F[i, v] for v in m.Set_VF)

    # 3. Monotonicity
    m.c_mono_1 = pyo.Constraint(m.I, rule=lambda m, i: get_vL(m, i) >= get_vTheta(m, i))
    m.c_mono_2 = pyo.Constraint(m.I, rule=lambda m, i: get_vTheta(m, i) >= get_vF(m, i))

    # 4. Physics (Safe Geometry + Discrete Speed)
    def physics_rule(m, i):
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        
        # Safe Geometry Logic
        d0_sq = (p_x[i]-Cx)**2 + (p_y[i]-Cy)**2
        d0_p_sq = (p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2
        
        safe_term = pyo.Expr_if(d0_sq - r**2 > 1e-6, d0_sq - r**2, 0.0)
        d_L = pyo.sqrt(safe_term + 1e-6)
        
        # d0 calculation for angles
        d0 = pyo.sqrt(d0_sq + 1e-6)
        
        term1 = r / d0
        term2 = (r**2 + d0_sq - d0_p_sq) / (2*r*d0)
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        theta_rad = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        # Time = Distance / DiscreteSpeed
        t_L = (d_L / get_vL(m, i)) * 3600
        t_turn = (d_theta / get_vTheta(m, i)) * 3600
        t_final = (d_final / get_vF(m, i)) * 3600
        
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final

    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)

    # 5. Separation with Slack (CRITICAL FIX)
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I: return pyo.Constraint.Skip
        # The slack allows the NLP subproblem to converge even if MILP guess is slightly off
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)

    # --- D. Objective ---
    # Penalty of 5000 ensures slack is only used if absolutely necessary
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 5000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)

    # --- E. Solve ---
    solver = SolverFactory('mindtpy')
    
    print("Running MindtPy (Robust)...")
    try:
        # Increase nlp_iteration_limit to give Ipopt more time
        results = solver.solve(m, mip_solver='glpk', nlp_solver='ipopt', 
                               tee=debug, 
                               iteration_limit=30, # Max outer iterations
                               nlp_iteration_limit=3000) # Max Ipopt iterations
        
    except Exception as e:
        print(f"Solver Crashed: {e}")
        return pd.DataFrame()

    # --- F. Safety Check Before Extraction ---
    # This prevents the "No value for uninitialized VarData" error
    if results.solver.termination_condition == TerminationCondition.infeasible:
        print("MindtPy declared problem INFEASIBLE.")
        return pd.DataFrame()
        
    # Check if a Primal solution actually exists
    # MindtPy sometimes returns 'optimal' status even if it only found a bound
    try:
        pyo.value(m.d[flight_ids[0]])
    except ValueError:
        print("MindtPy failed to load a valid Primal solution (likely cycling).")
        return pd.DataFrame()

    # --- G. Extract ---
    res = []
    for i in m.I:
        # Extract selected speeds
        vL_val = sum(v for v in m.Set_VL if pyo.value(m.z_L[i, v]) > 0.5)
        vTheta_val = sum(v for v in m.Set_Vtheta if pyo.value(m.z_theta[i, v]) > 0.5)
        vF_val = sum(v for v in m.Set_VF if pyo.value(m.z_F[i, v]) > 0.5)
        
        res.append({
            'aircraft_id': i,
            'd_i': pyo.value(m.d[i]),
            'v_L': vL_val,
            'v_theta': vTheta_val,
            'v_f': vF_val,
            'arrival_time': pyo.value(m.t[i]),
            'slack': pyo.value(m.slack[i])
        })
        
    return pd.DataFrame(res)

# --- Instructions to Run ---
# Uncomment the line below ONLY if you have 'glpk' installed alongside 'ipopt'
df_minlp = solve_exact_discrete_minlp(df_arrivals, debug=True)

# Calculate achieved separation
df_minlp = pd.merge(df_arrivals, df_minlp, on='aircraft_id')
df_minlp['prev_arrival'] = df_minlp['arrival_time'].shift(1)
df_minlp['actual_separation'] = df_minlp['arrival_time'] - df_minlp['prev_arrival']

display(df_minlp)

NameError: name 'solve_exact_discrete_minlp' is not defined

# Setup III: Co-optimize $d_i$, segments speed, and the arrival sequence (Do Not Follow FCFS Rule)
 - This is the most computationally complex formulation: Mixed-Integer Nonlinear Programming (MINLP).

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd
import itertools

# ==========================================
# 9. MINLP SEQUENCING + TRAJECTORY
#    Variables: d_i, Speeds (Continuous), Sequence (Binary)
# ==========================================

def solve_sequencing_minlp(df_arrivals, debug=False):
    """
    Implements Setup III: Co-optimize Sequence, Speed, and d_i.
    Uses Big-M formulation for disjunctive constraints.
    """
    
    # --- Limit Problem Size for Demo ---
    # MINLP scales factorially. We limit to first 5 aircraft for this test.
    df_subset = df_arrivals.copy() 
    print(f"Solving Optimal Sequence for {len(df_subset)} aircraft...")
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_subset['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Create Set of Pairs (i, j) where i < j (to avoid double counting)
    # We will decide sequence for every unique pair
    m.Pairs = pyo.Set(initialize=[(i, j) for i in flight_ids for j in flight_ids if i < j])
    
    p_tau = df_subset.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_subset.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_subset.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_subset.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_subset.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # Big-M Parameter (Must be large enough to make constraint inactive when y=0)
    # But not too large to cause numerical errors. 
    # Max logical separation ~ 1 hour (3600s)
    M = 5000.0 

    # --- B. Decision Variables ---
    
    # Trajectory Variables
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0, 10000), initialize=lambda m, i: p_tau[i] + 600)
    
    # Continuous Speed Variables (Section 2.3 logic)
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
    
    # SEQUENCE VARIABLE (Binary)
    # y[i,j] = 1 if i arrives BEFORE j
    # y[i,j] = 0 if j arrives BEFORE i
    m.y = pyo.Var(m.Pairs, domain=pyo.Binary)
    
    # Makespan Variable (The scalar we want to minimize)
    m.makespan = pyo.Var(domain=pyo.NonNegativeReals)

    # --- C. Trajectory Physics Constraints (Same as before) ---
    def calc_distances_expr(m, i):
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        d0 = pyo.sqrt((p_x[i]-Cx)**2 + (p_y[i]-Cy)**2 + 1e-6)
        d0_p = pyo.sqrt((p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2)
        d_L = pyo.sqrt(d0**2 - r**2 + 1e-6)
        theta1 = pyo.acos(r/(d0+1e-6))
        theta2 = pyo.acos((r**2 + d0**2 - d0_p**2)/(2*r*(d0+1e-6)))
        theta_rad = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        return d_L, r * theta_rad, m.d[i]

    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        # Constraint: Arrival Time must equal Entry + Travel
        # Note: We use inequality (>=) here to allow "Delay" at the entry point 
        # if optimization requires holding, though usually equality is preferred. 
        # For this rigorous formulation, we stick to equality (physics).
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # Speed Monotonicity
    m.c_speed1 = pyo.Constraint(m.I, rule=lambda m, i: m.v_L[i] >= m.v_theta[i])
    m.c_speed2 = pyo.Constraint(m.I, rule=lambda m, i: m.v_theta[i] >= m.v_f[i])

    # --- D. Big-M Sequencing Constraints ---
    
    def seq_rule_1(m, i, j):
        # Constraint: If y[i,j]=1 (i before j), then t[j] >= t[i] + Sep
        # If y[i,j]=0, the -M term makes this constraint trivial (-5000), so it's ignored
        return m.t[j] >= m.t[i] + T_SEP_SEC - M * (1 - m.y[i, j])
    m.c_seq_1 = pyo.Constraint(m.Pairs, rule=seq_rule_1)
    
    def seq_rule_2(m, i, j):
        # Constraint: If y[i,j]=0 (j before i), then t[i] >= t[j] + Sep
        return m.t[i] >= m.t[j] + T_SEP_SEC - M * (m.y[i, j])
    m.c_seq_2 = pyo.Constraint(m.Pairs, rule=seq_rule_2)

    # --- E. Objective: Minimize Makespan ---
    
    # Constrain makespan variable to be >= all arrival times
    def makespan_def(m, i):
        return m.makespan >= m.t[i]
    m.c_makespan = pyo.Constraint(m.I, rule=makespan_def)
    
    m.obj = pyo.Objective(expr=m.makespan, sense=pyo.minimize)
    
    # --- F. Solve with MindtPy ---
    solver = SolverFactory('mindtpy')
    
    print("Starting MINLP Solver (This may take time)...")
    try:
        # Using 'oa' (Outer Approximation) is generally good for convex NLPs
        # time_limit ensures it doesn't hang forever on tutorial
        results = solver.solve(m, mip_solver='glpk', nlp_solver='ipopt', tee=debug, time_limit=600)
    except Exception as e:
        print(f"Solver Error: {e}")
        return pd.DataFrame()

    # --- G. Extract Results ---
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time': pyo.value(m.t[i]),
            'v_L': pyo.value(m.v_L[i]),
            'v_theta': pyo.value(m.v_theta[i]),
            'v_f': pyo.value(m.v_f[i]),
            'entry_time': p_tau[i]
        })
        
    return pd.DataFrame(res).sort_values('arrival_time')

# --- Run ---
# Note: Ensure you have 'glpk' installed (conda install glpk)
# If not, this cell will throw an error.
df_seq_results = solve_sequencing_minlp(df_arrivals, debug=True)

if not df_seq_results.empty:
    print("\nOptimal Sequence Found:")
    # Calculate the sequence shifts
    df_seq_results['seq_rank'] = range(1, len(df_seq_results)+1)
    df_seq_results['original_id_rank'] = df_seq_results['aircraft_id'].rank()
    
    display(df_seq_results[['seq_rank', 'aircraft_id', 'entry_time', 'arrival_time', 'v_L', 'v_theta', 'v_f']])
    
    # Did the order change?
    if not df_seq_results['seq_rank'].equals(df_seq_results['original_id_rank']):
        print("NOTICE: The solver re-sequenced the aircraft to improve efficiency!")
    else:
        print("NOTICE: FCFS was found to be the optimal sequence.")

Starting MindtPy version 1.0.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe1472b0>
call_before_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe1472e0>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe147310>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe147340>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: f

Solving Optimal Sequence for 23 aircraft...
Starting MINLP Solver (This may take time)...
deprecated. Either specify deactivated Blocks as targets to activate them if
transforming them is the desired behavior.  (deprecated in 6.9.3) (called from
/home/yp6443/miniconda3/envs/tracon/lib/python3.10/site-
packages/pyomo/core/base/transformation.py:77)


         -       Relaxed NLP           4292.67            inf        4292.67      nan%      0.38
         1              MILP           4292.67            inf        4292.67      nan%      0.51
         1         Fixed NLP        Infeasible            inf        4292.67      nan%      0.73
         1   Feasibility NLP           18.4237            inf        4292.67      nan%      0.85
MILP solver reported feasible solution, but not guaranteed to be optimal.
         2              MILP           4327.87            inf        4292.67      nan%      0.93
*        2         Fixed NLP           4315.03        4315.03        4292.67     0.52%      1.09
MIP main problem is infeasible. Problem may have no more feasible binary configurations.
MindtPy exiting due to MILP main problem infeasibility.
 Primal integral          :    0.0000 
 Dual integral            :    0.0000 
 Primal-dual gap integral :    0.0000 



Optimal Sequence Found:


,seq_rank,aircraft_id,entry_time,arrival_time,v_L,v_theta,v_f
4,1,5,337.280896,807.377713,231.769360,179.183564,150.626124
1,2,2,90.378260,943.681531,220.307773,166.907774,141.255693
7,3,8,1155.105225,1627.600441,231.535556,179.033680,150.558924
6,4,7,1129.459820,1898.290257,247.595839,197.251639,154.294014
0,5,1,69.404223,1969.477271,183.303309,139.648225,132.462820
3,6,4,230.810359,2044.224754,193.704780,155.307973,137.438435
2,7,3,193.250072,2120.173764,182.048274,136.274242,131.584512
5,8,6,352.635355,2243.004304,187.656639,148.638115,135.112005
14,9,15,2701.287850,3100.116074,245.345991,191.953197,154.777546
10,10,11,2147.057566,3173.417089,192.675026,143.978554,132.737083


NOTICE: The solver re-sequenced the aircraft to improve efficiency!


### with slack variables

In [ ]:
# ==========================================
# 9. ROBUST MINLP SEQUENCING
#    Variables: d_i, Speeds (Continuous), Sequence (Binary), Slack
# ==========================================

def solve_sequencing_minlp_robust(df_arrivals, debug=False):
    """
    Setup 3: Co-optimize Sequence, Speed (Continuous), and d_i.
    Includes Slack variables to prevent solver failure on bad sequences.
    """
    
    # --- LIMIT DATA SIZE ---
    # Sequencing is Factorial complexity (N!). 
    # Start with 5-7 aircraft. 10+ requires commercial solvers (Gurobi).
    df_subset = df_arrivals.head(5).copy() 
    print(f"Optimizing Sequence for first {len(df_subset)} aircraft...")
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_subset['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Pairs (i, j) for sequencing logic (i < j to avoid duplicates)
    m.Pairs = pyo.Set(initialize=[(i, j) for i in flight_ids for j in flight_ids if i < j])
    
    p_tau = df_subset.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_subset.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_subset.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_subset.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_subset.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # Big-M: Sufficiently large to turn off constraints (e.g., 1 hour)
    M = 3600.0 

    # --- B. Decision Variables ---
    
    # 1. Physics Variables
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0, 20000), initialize=lambda m, i: p_tau[i] + 600)
    
    # 2. Continuous Speed Variables (Setup 2.3 logic)
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
    
    # 3. Sequencing Binary: y[i,j]=1 if i before j
    m.y = pyo.Var(m.Pairs, domain=pyo.Binary)
    
    # 4. Pairwise Slack: Allows violation of separation for specific pairs
    # If the solver picks a bad sequence, it uses this slack variable.
    m.slack = pyo.Var(m.Pairs, domain=pyo.NonNegativeReals, initialize=0.0)
    
    # 5. Makespan (Scalar)
    m.makespan = pyo.Var(domain=pyo.NonNegativeReals, initialize=p_tau[flight_ids[-1]] + 1000)

    # --- C. Physics Constraints (Safe Geometry) ---
    def physics_rule(m, i):
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        
        # Safe Geometry (Prevent sqrt crash)
        d0_sq = (p_x[i]-Cx)**2 + (p_y[i]-Cy)**2
        d0_p_sq = (p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2
        
        safe_term = pyo.Expr_if(d0_sq - r**2 > 1e-6, d0_sq - r**2, 0.0)
        d_L = pyo.sqrt(safe_term + 1e-6)
        
        d0 = pyo.sqrt(d0_sq + 1e-6)
        theta1 = pyo.acos(r/d0)
        theta2 = pyo.acos((r**2 + d0_sq - d0_p_sq)/(2*r*d0))
        theta_rad = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        
        d_theta = r * theta_rad
        d_final = m.d[i]

        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # Speed Monotonicity
    m.c_speed1 = pyo.Constraint(m.I, rule=lambda m, i: m.v_L[i] >= m.v_theta[i])
    m.c_speed2 = pyo.Constraint(m.I, rule=lambda m, i: m.v_theta[i] >= m.v_f[i])

    # --- D. Big-M Sequencing with Slack ---
    
    # Case 1: i comes before j (y=1)
    # Constraint: t[j] >= t[i] + Sep - Slack - BigM*(0)
    def seq_rule_1(m, i, j):
        return m.t[j] >= m.t[i] + T_SEP_SEC - m.slack[i, j] - M * (1 - m.y[i, j])
    m.c_seq_1 = pyo.Constraint(m.Pairs, rule=seq_rule_1)
    
    # Case 2: j comes before i (y=0)
    # Constraint: t[i] >= t[j] + Sep - Slack - BigM*(0)
    # Note: We use the SAME slack variable for the pair (i,j) to save memory
    def seq_rule_2(m, i, j):
        return m.t[i] >= m.t[j] + T_SEP_SEC - m.slack[i, j] - M * (m.y[i, j])
    m.c_seq_2 = pyo.Constraint(m.Pairs, rule=seq_rule_2)

    # --- E. Objective ---
    # Minimize Makespan + Heavy Penalty for using Slack
    def makespan_def(m, i):
        return m.makespan >= m.t[i]
    m.c_makespan = pyo.Constraint(m.I, rule=makespan_def)
    
    # Penalty = 10,000. 
    # Logic: It is much better to delay the makespan by 1 hour than to have 1 second of safety violation.
    m.obj = pyo.Objective(expr=m.makespan + 10000 * sum(m.slack[i, j] for i, j in m.Pairs), sense=pyo.minimize)
    
    # --- F. Solve ---
    print("Running MindtPy (Sequencing)...")
    solver = SolverFactory('mindtpy')
    
    try:
        # Strategy 'oa' (Outer Approximation) usually works best for these physics problems
        # time_limit is important so it doesn't hang indefinitely
        results = solver.solve(m, mip_solver='glpk', nlp_solver='ipopt', 
                               tee=debug, 
                               strategy='oa',
                               time_limit=600) 
    except Exception as e:
        print(f"Solver Error: {e}")
        return pd.DataFrame()

    # --- G. Extract ---
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'entry_time': p_tau[i],
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time': pyo.value(m.t[i]),
            'v_L': pyo.value(m.v_L[i]),
            'v_f': pyo.value(m.v_f[i])
        })
    
    # Check total slack used
    total_slack = sum(pyo.value(m.slack[i, j]) for i, j in m.Pairs)
    print(f"\nTotal Sequencing Slack Used: {total_slack:.2f} s")
    if total_slack > 0.1:
        print("WARNING: The solver could not find a strictly valid sequence (capacity overloaded).")
        
    return pd.DataFrame(res).sort_values('arrival_time')

# --- Run ---
# Note: Ensure you have 'glpk' installed (conda install glpk)
# If not, this cell will throw an error.
df_seq_results = solve_sequencing_minlp(df_arrivals, debug=True)

if not df_seq_results.empty:
    print("\nOptimal Sequence Found:")
    # Calculate the sequence shifts
    df_seq_results['seq_rank'] = range(1, len(df_seq_results)+1)
    df_seq_results['original_id_rank'] = df_seq_results['aircraft_id'].rank()
    
    display(df_seq_results[['seq_rank', 'aircraft_id', 'entry_time', 'arrival_time', 'v_L', 'v_theta', 'v_f']])
    
    # Did the order change?
    if not df_seq_results['seq_rank'].equals(df_seq_results['original_id_rank']):
        print("NOTICE: The solver re-sequenced the aircraft to improve efficiency!")
    else:
        print("NOTICE: FCFS was found to be the optimal sequence.")

Starting MindtPy version 1.0.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe1472b0>
call_before_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe1472e0>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe147310>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x6ffdfe147340>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: f

Solving Optimal Sequence for 23 aircraft...
Starting MINLP Solver (This may take time)...


         -       Relaxed NLP           4292.67            inf        4292.67      nan%      0.35
         1              MILP           4292.67            inf        4292.67      nan%      0.43
         1         Fixed NLP        Infeasible            inf        4292.67      nan%      0.65
         1   Feasibility NLP           18.4237            inf        4292.67      nan%      0.78
MILP solver reported feasible solution, but not guaranteed to be optimal.
         2              MILP           4327.87            inf        4292.67      nan%      0.86
*        2         Fixed NLP           4315.03        4315.03        4292.67     0.52%      1.02
MIP main problem is infeasible. Problem may have no more feasible binary configurations.
MindtPy exiting due to MILP main problem infeasibility.
 Primal integral          :    0.0000 
 Dual integral            :    0.0000 
 Primal-dual gap integral :    0.0000 



Optimal Sequence Found:


,seq_rank,aircraft_id,entry_time,arrival_time,v_L,v_theta,v_f
4,1,5,337.280896,807.377713,231.769360,179.183564,150.626124
1,2,2,90.378260,943.681531,220.307773,166.907774,141.255693
7,3,8,1155.105225,1627.600441,231.535556,179.033680,150.558924
6,4,7,1129.459820,1898.290257,247.595839,197.251639,154.294014
0,5,1,69.404223,1969.477271,183.303309,139.648225,132.462820
3,6,4,230.810359,2044.224754,193.704780,155.307973,137.438435
2,7,3,193.250072,2120.173764,182.048274,136.274242,131.584512
5,8,6,352.635355,2243.004304,187.656639,148.638115,135.112005
14,9,15,2701.287850,3100.116074,245.345991,191.953197,154.777546
10,10,11,2147.057566,3173.417089,192.675026,143.978554,132.737083


NOTICE: The solver re-sequenced the aircraft to improve efficiency!


## Try the time-window decomposition approach
Instead of solving everything at once, we split the problem into two physically coupled steps:
- Step 1: Pre-calculate Feasibility Windows (The Physics)For each aircraft $i$, calculate the Earliest Possible Arrival Time ($E_i$) by flying the shortest path at maximum speed.Calculate the Latest Possible Arrival Time ($L_i$) by flying the longest path at minimum speed.Now, we know that aircraft $i$ can physically arrive anywhere in the interval $[E_i, L_i]$.
- Step 2: Solve the Sequencing (The Scheduling MILP)This becomes a pure Linear Programming problem (MILP). No square roots. No geometry.We just determine arrival times $t_i$ such that $E_i \le t_i \le L_i$ and everyone is separated by 64 seconds.This solves in milliseconds.
- Step 3: Trajectory Recovery (Post-processing)Once the MILP gives us the optimal time $t^*_i$, we simply find the specific $d_i$ and speed $v$ that achieves that time.

In [ ]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# --- Helper: Geometry Function (Python only, no Pyomo variables) ---
def calculate_flight_time(d_i, v_L, v_theta, v_f, row, r, x_faf, y_faf):
    """Calculates flight time in seconds given fixed inputs."""
    # Geometry Inputs
    y_off = r if row['is_north'] else -r
    Cx, Cy = x_faf - d_i, y_faf + y_off
    
    d0_sq = (row['x_entry']-Cx)**2 + (row['y_entry']-Cy)**2
    if d0_sq <= r**2: return 99999.0 # Infeasible geometry
    
    d_L = np.sqrt(d0_sq - r**2)
    d0 = np.sqrt(d0_sq)
    d0_p_sq = (row['x_entry']-(x_faf-d_i))**2 + (row['y_entry']-y_faf)**2
    
    # Angles
    term1 = np.clip(r/d0, -1, 1)
    term2 = np.clip((r**2 + d0_sq - d0_p_sq)/(2*r*d0), -1, 1)
    
    theta1 = np.arccos(term1)
    theta2 = np.arccos(term2)
    
    if row['long_arc']:
        theta_rad = 2*np.pi - (theta1+theta2)
    else:
        theta_rad = theta2 - theta1
        
    dist_turn = r * theta_rad
    
    # Time Calc
    t_sec = (d_L/v_L + dist_turn/v_theta + d_i/v_f) * 3600
    return t_sec

def calculate_time_windows(df_arrivals):
    """
    Adds 'min_time' and 'max_time' to the dataframe.
    """
    r = 6
    x_faf = -5.1
    y_faf = 0.0
    
    results = []
    
    for idx, row in df_arrivals.iterrows():
        # Earliest Arrival: Max Speed, Min Distance
        # V_max sets: L=250, Theta=200, F=200. d_i=0
        t_min_flight = calculate_flight_time(0.0, 250.0, 200.0, 200.0, row, r, x_faf, y_faf)
        t_earliest = row['entry_time'] + t_min_flight
        
        # Latest Arrival: Min Speed, Max Distance
        # V_min sets: L=180, Theta=130, F=130. d_i=20
        t_max_flight = calculate_flight_time(20.0, 180.0, 130.0, 130.0, row, r, x_faf, y_faf)
        t_latest = row['entry_time'] + t_max_flight
        
        results.append({
            'aircraft_id': row['aircraft_id'],
            'min_arrival': t_earliest,
            'max_arrival': t_latest
        })
        
    return pd.merge(df_arrivals, pd.DataFrame(results), on='aircraft_id')

# Calculate the windows
df_windows = calculate_time_windows(df_arrivals)
print("Time Windows Calculated:")
display(df_windows[['aircraft_id', 'entry_time', 'min_arrival', 'max_arrival']].head())

Time Windows Calculated:


,aircraft_id,entry_time,min_arrival,max_arrival
0,1,69.404223,778.730529,2046.272237
1,2,90.378260,473.351537,1170.804246
2,3,193.250072,902.576378,2170.118086
3,4,230.810359,986.411726,2280.024205
4,5,337.280896,720.254172,1417.706881


In [ ]:
def solve_linear_sequencing(df_windows, debug=False):
    print(f"Solving Linear Sequencing for {len(df_windows)} aircraft...")
    
    m = pyo.ConcreteModel()
    
    ids = df_windows['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=ids)
    m.Pairs = pyo.Set(initialize=[(i, j) for i in ids for j in ids if i < j])
    
    # Parameters: The Time Windows
    p_min = df_windows.set_index('aircraft_id')['min_arrival'].to_dict()
    p_max = df_windows.set_index('aircraft_id')['max_arrival'].to_dict()
    
    # Variables
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals) # Arrival Time
    m.y = pyo.Var(m.Pairs, domain=pyo.Binary)       # Sequence
    m.makespan = pyo.Var(domain=pyo.NonNegativeReals)
    
    # Slack is still useful for dense traffic, but now it's linear!
    m.slack = pyo.Var(m.Pairs, domain=pyo.NonNegativeReals, initialize=0.0)
    
    M = 10000.0 # Big-M
    
    # --- Constraints ---
    
    # 1. Physics Window Constraints (Hard bounds)
    def time_window_rule(m, i):
        return (p_min[i], m.t[i], p_max[i])
    m.c_window = pyo.Constraint(m.I, rule=time_window_rule)
    
    # 2. Separation Logic (Linear Big-M)
    def seq_1(m, i, j):
        # If y[i,j]=1 (i first), t[j] >= t[i] + 64
        return m.t[j] >= m.t[i] + 64.0 - m.slack[i,j] - M*(1-m.y[i,j])
    m.c_seq_1 = pyo.Constraint(m.Pairs, rule=seq_1)

    def seq_2(m, i, j):
        # If y[i,j]=0 (j first), t[i] >= t[j] + 64
        return m.t[i] >= m.t[j] + 64.0 - m.slack[i,j] - M*(m.y[i,j])
    m.c_seq_2 = pyo.Constraint(m.Pairs, rule=seq_2)
    
    # 3. Objective
    def makespan_rule(m, i): return m.makespan >= m.t[i]
    m.c_makespan = pyo.Constraint(m.I, rule=makespan_rule)
    
    m.obj = pyo.Objective(expr=m.makespan + 5000*sum(m.slack[i,j] for i,j in m.Pairs), sense=pyo.minimize)
    
    # Solve with GLPK or CBC (Pure Integer Linear Solver)
    # We do NOT need Ipopt here.
    solver = SolverFactory('glpk') # or 'cbc'
    res = solver.solve(m, tee=debug)
    
    # Extract Results
    schedule = []
    for i in m.I:
        schedule.append({
            'aircraft_id': i,
            'target_time': pyo.value(m.t[i])
        })
    return pd.DataFrame(schedule).sort_values('target_time')

# Run the Linear Sequencer
# Note: You can pass ALL 29 aircraft here, GLPK handles 29 planes easily.
df_schedule_target = solve_linear_sequencing(df_windows, debug=True)
display(df_schedule_target.head())

Solving Linear Sequencing for 23 aircraft...


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpy_o9cpe3.glpk.raw --wglp /tmp/tmpqvjr290_.glpk.glp --cpxlp
 /tmp/tmpe4ww4yom.pyomo.lp
Reading problem data from '/tmp/tmpe4ww4yom.pyomo.lp'...
/tmp/tmpe4ww4yom.pyomo.lp:4635: warning: lower bound of variable 'x303' redefined
/tmp/tmpe4ww4yom.pyomo.lp:4635: warning: upper bound of variable 'x303' redefined
575 rows, 530 columns, 2116 non-zeros
253 integer variables, all of which are binary
4888 lines were read
Writing problem data to '/tmp/tmpqvjr290_.glpk.glp'...
4330 lines were written
GLPK Integer Optimizer 5.0
575 rows, 530 columns, 2116 non-zeros
253 integer variables, all of which are binary
Preprocessing...
369 constraint coefficient(s) were reduced
382 rows, 530 columns, 1502 non-zeros
253 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.177e+03  ratio =  5.177e+03
GM: min|aij| =  2.318e-01  max|aij| =  4.314e+00  ratio =  1.861e+01
EQ: min|ai

,aircraft_id,target_time
4,5,1106.804246
1,2,1170.804246
7,8,1790.272237
5,6,1854.272237
3,4,1918.272237


In [ ]:
def recover_trajectory(df_original, df_targets):
    """
    Given the Target Time, find d_i and Speeds.
    Minimize (Path Extension) + (Speed Reduction).
    """
    combined = pd.merge(df_original, df_targets, on='aircraft_id')
    final_res = []
    
    print("Recovering Trajectories...")
    
    for idx, row in combined.iterrows():
        # Setup a mini-optimization for ONE aircraft
        m = pyo.ConcreteModel()
        
        # Geometry Params
        r = 10.0
        x_faf, y_faf = -5.1, 0.0
        p_north = row['is_north']
        p_long = row['long_arc']
        
        # Variables
        m.d = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
        m.v_L = pyo.Var(domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
        m.v_theta = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
        m.v_f = pyo.Var(domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
        
        # Physics Expression (Same as before)
        def physics_rule(m):
            y_off = r if p_north else -r
            Cx, Cy = x_faf - m.d, y_faf + y_off
            
            d0_sq = (row['x_entry']-Cx)**2 + (row['y_entry']-Cy)**2
            d0_p_sq = (row['x_entry']-(x_faf-m.d))**2 + (row['y_entry']-y_faf)**2
            
            # We assume valid geometry because Phase 1 checked it
            d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
            d0 = pyo.sqrt(d0_sq + 1e-6)
            
            theta1 = pyo.acos(r/d0)
            theta2 = pyo.acos((r**2 + d0_sq - d0_p_sq)/(2*r*d0))
            theta_rad = (2*3.14159 - (theta1+theta2)) if p_long else (theta2-theta1)
            
            t_sec = (d_L/m.v_L + (r*theta_rad)/m.v_theta + m.d/m.v_f) * 3600
            
            # Constraint: Must hit the target time exactly
            return row['entry_time'] + t_sec == row['target_time']
            
        m.c_phys = pyo.Constraint(rule=physics_rule)
        
        # Monotonicity
        m.c_mono1 = pyo.Constraint(expr=m.v_L >= m.v_theta)
        m.c_mono2 = pyo.Constraint(expr=m.v_theta >= m.v_f)
        
        # Objective: Efficiency
        # Maximize Speeds (Minimize negative speed) + Minimize d_i
        m.obj = pyo.Objective(expr=m.d - 0.01*(m.v_L + m.v_theta + m.v_f), sense=pyo.minimize)
        
        solver = SolverFactory('ipopt')
        solver.options['print_level'] = 0
        solver.solve(m)
        
        final_res.append({
            'aircraft_id': row['aircraft_id'],
            'arrival_time': row['target_time'],
            'd_i': pyo.value(m.d),
            'v_L': pyo.value(m.v_L),
            'v_theta': pyo.value(m.v_theta),
            'v_f': pyo.value(m.v_f)
        })
        
    return pd.DataFrame(final_res).sort_values('arrival_time')

# Run Reconstruction
df_final_sequenced = recover_trajectory(df_arrivals, df_schedule_target)
display(df_final_sequenced)

Recovering Trajectories...


ValueError: Cannot load a SolverResults object with bad status: error